# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.21.230828.2-preview"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();

Let's define a prompt outlining a dialogue chat bot.

In [ ]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.7,
        TopP = 0.5,
    }
};

Register your semantic function

In [ ]:
var promptTemplate = new PromptTemplate(skPrompt, promptConfig, kernel);
var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", functionConfig);

Initialize your context

In [ ]:
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

Chat with the Bot

In [ ]:
var userInput = "Hi, I'm looking for book suggestions";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

Update the history with the output and set this as the new input value for the next request

In [ ]:
history += $"\nUser: {userInput}\nMelody: {bot_answer}\n";
context.Variables.Update(history);

Console.WriteLine(context);

Keep Chatting!

In [ ]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nMelody: {answer}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

In [ ]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

In [ ]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

In [ ]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");

In [ ]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [ ]:
Console.WriteLine(context.Variables["history"]);